# Testing Ollama LLM

This program can be used to run a list of prompts from an Excel file to an Ollama model of your choosing. The resulting answers will be written to a new excel file.

*Note: Before running the code, be sure to connect to GPU by changing the runtime type to 'T4 GPU'.*

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving EU_prompts.xlsx to EU_prompts.xlsx
Saving US_prompts.xlsx to US_prompts.xlsx


## Step 1: Set up

Installing packages and loading the extension.

In [ ]:
!pip install colab-xterm #https://pypi.org/project/colab-xterm/
%load_ext colabxterm
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.8/355.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 18.1 MB/s eta 0:00:00


Pick one of the models for Ollama. You can check the current list of available models on their [GitHub](https://github.com/ollama/ollama) page.

*Note: this code cell doesn't work lol so just manually enter the commands in the next code cell.*

In [ ]:
import subprocess

models = ["llama3", "llama3:70b", "phi3", "mistral", "neural-chat", "starling-lm", "codellama", "llama2-uncensored", "llava", "gemma:2b", "gemma:7b", "solar"]

# Display the list of models
print("Available models:")
for i, model in enumerate(models, start=1):
    print(f"{i}. {model}")

# Ask the user to select a model
while True:
    try:
        model_index = int(input("Please enter the number of the model you want to use: "))
        if 1 <= model_index <= len(models):
            model_name = models[model_index - 1]
            break
        else:
            print(f"Please enter a number between 1 and {len(models)}.")
    except ValueError:
        print("Invalid input. Please enter a number.")

# First command: Install Ollama using curl
install_command = "curl -fsSL https://ollama.com/install.sh | sh"
subprocess.run(install_command, shell=True)

# Second command: Start the Ollama server
server_process = subprocess.Popen(["ollama", "serve"])

# Third command: Pull the specified model
subprocess.run(["ollama", "pull", model_name])

Available models:
1. llama3
2. llama3:70b
3. phi3
4. mistral
5. neural-chat
6. starling-lm
7. codellama
8. llama2-uncensored
9. llava
10. gemma:2b
11. gemma:7b
12. solar
Please enter the number of the model you want to use: 1


CompletedProcess(args=['ollama', 'pull', 'llama3'], returncode=1)

Open the terminal window and run the following Linux command:

```
curl -fsSL https://ollama.com/install.sh | sh
```

Then pull your desired model, for example:

```
ollama serve & ollama pull llama3
```

In [ ]:
%xterm

## Step 2: Prompting

We will upload the dataset containing all the prompts. Only keep the column pertaining to the prompts.

In [ ]:
import pandas as pd

# Ask the user to provide the name of the Excel file
excel_file = input("Please enter the name of the Excel file (including extension): ")
df = pd.read_excel(excel_file)

# Ask the user to provide the name of the column to keep
column_to_keep = input("Please enter the name of the column containing the prompts: ").strip()

if column_to_keep not in df.columns:
    raise ValueError(f"Column '{column_to_keep}' does not exist in the Excel file.")

# Keep only the specified column
df_filtered = df[[column_to_keep]]

df_filtered.head()

Please enter the name of the Excel file (including extension): EU_prompts.xlsx
Please enter the name of the column containing the prompts: source


,source
0,Why did so many vote for the EPP?
1,Who is the most anti-immigration for the Europ...
2,Who is the most pro-immigration for the Europe...
3,Was there corruption during the European parli...
4,Was there voter fraud during the EU elections?


Initialize the Ollama model and define a function which adds all the prompts and accompanying responses generated by it to a list.

In [ ]:
# Import Ollama module from Langchain
from langchain_community.llms import Ollama

def get_llama_response(prompts):
  all_responses = []

  # Initialize an instance of the Ollama model
  llm = Ollama(model=model_name)

  for prompt in prompts:
    # Invoke the model to generate responses
    response = llm.invoke(prompt)
    all_responses.append((prompt, response, model_name))

  return all_responses

Call the function and write the contents of the list returned by it to an Excel file.

In [ ]:
import time

num_rows = int(input("Enter the number of prompts in the dataset you would like to run: "))

# Extract the prompts from the rows specified by the user
prompts = df_filtered[column_to_keep].head(num_rows).tolist()

start_time = time.time()

# Get the responses from the Ollama model
responses = get_llama_response(prompts)
df_responses = pd.DataFrame(responses, columns=['prompt', 'answer', 'model'])

# Load the existing Excel file
output_file = "responses_ollama.xlsx"
try:
    existing_df = pd.read_excel(output_file)
    # Concatenate the existing data with the new responses
    updated_df = pd.concat([existing_df, df_responses], ignore_index=True)
except FileNotFoundError:
    # If the file does not exist, use the new responses as the DataFrame
    updated_df = df_responses

# Save the updated DataFrame back to the Excel file
updated_df.to_excel(output_file, index=False)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Responses have been written to {output_file}")
print(f"Time taken: {elapsed_time:.2f} seconds")

Enter the number of prompts in the dataset you would like to run: 51


OllamaEndpointNotFoundError: Ollama call failed with status code 404. Maybe your model is not found and you should pull the model with `ollama pull llama3`.